In [1]:
from __future__ import absolute_import, division, print_function

from psychopy import visual, core, event, gui, data
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Experiment Design
## Block structure
1. 4 Blocks of Template Trials
    - 8 Directions. 
    - No rotation
    - No Go/NoGo
2. 4 Blocks of Rotated Trials
    - 4 Directions.
    - 90$^{\circ}$ rotation
    - Go/NoGo
3. 4 Blocks of Transfer Trials
    - 4 Directions.
    - No Rotations
    - Go NoGo


## Trial Structure
1. Template Trials:
    - Target and Fixation Appear
    - Drag and rop fixation to target
    - Trial ends with the drop. Next set of target and fixations appear after 2.5-5 seconds later, centered on 3.5 seconds
2. Rotation Trials:
    - Fixation Appears.
    - Target Appears after 

## Experiment Prompt

In [31]:
myDlg = gui.Dlg(title = 'Visuomotor Adaptation')
myDlg.addField('Participant Number: ')
ok_data = myDlg.show()  # show dialog and wait for OK or Cancel
if myDlg.OK:  # or if ok_data is not None
    print(ok_data)
else:
    print('user cancelled')

['2000']


## 

## Create Target Stimuli

## Scoreboard

In [82]:
def display_scoreboard(window, text):
    score_text = visual.TextBox(window, 
               text, 
               font_size=21,
               font_color=[-1,-1,1], 
                size = (0.75, 1),
               pos=(0.0,0.0), 
               grid_horz_justification='center',
                align_horz='center',
               units='norm')
    score_text.draw()
    my_mouse = event.Mouse(visible=False,newPos=[0,0],win=window)
    window.update()
    while True:
        buttons = my_mouse.getPressed()
        if buttons == [1, 0, 0]:
            break


# Template Trials

### Target Stimuli

### Experiment display

In [59]:
def template_blocks(num_blocks, num_trials, rotation):
    thetas = np.repeat(np.array([0, np.pi/4, np.pi/2, 3*np.pi/4, 2*np.pi, 5*np.pi/4, 3*np.pi/2, 7*np.pi/4]), 2)
    target_position_thetas = np.array([np.random.permutation(thetas), np.random.permutation(thetas), np.random.permutation(thetas), np.random.permutation(thetas)])
    target_angle_pos = np.ravel(target_position_thetas)
    target_pos_radius = 0.75
    target_xpos = 0.42*np.cos(target_angle_pos)
    target_ypos = 0.75*np.sin(target_angle_pos)
    
    # Create a window to draw in
    my_win = visual.Window((1920, 1080), allowGUI=True)
    
  
    #Center fixation
    fixation = visual.Circle(my_win, edges=64, size = (0.03, 0.05), pos = [0, 0], fillColor = [0.5, 0.5, 0.5], lineColor = [0.5, 0.5, 0.5])

    #Initialize Variables to return
    rotated_traj_x = np.empty((num_blocks, num_trials), dtype = object)
    rotated_traj_y = np.empty((num_blocks, num_trials), dtype = object)
    initial_time = np.zeros((num_blocks, num_trials))
    movement_time = np.zeros((num_blocks, num_trials))

    for block in range(num_blocks):
        display_scoreboard(my_win, 'Your score: 10 \n Tap Stylus to continue')
        for trial in range(num_trials):
            traj_x = list()
            traj_y = list()

            fixation.draw()
            my_win.update()
            #isi
            core.wait(np.random.normal(3.5, 1))
            #Go Target
            target = visual.Circle(my_win, edges=64, size = (0.1, 0.17), pos = [target_xpos[trial], target_ypos[trial]], fillColor = [1, -1, -1], lineColor = [1, -1, -1])
            target.draw()            
            my_win.update()
            #Wait for color change
#            core.wait(np.random.uniform())
#            #Change fixation color
            fixation.fillColor = [-0.5, 1, 0.9] 
            fixation.lineColor = [-0.5, 1, 0.9]
            fixation.draw()
            target.draw()
            #Initialize mouse object
            my_mouse = event.Mouse(visible=False,newPos=[0,0],win=my_win)
            my_win.update()
            clock_it = core.Clock()
            clock_mt = core.Clock()
            first_flag = True
            while True:
                #Get current mouse position
                mouse_x, mouse_y = my_mouse.getPos()

                if my_mouse.getPressed()[0]:
                    if first_flag:
                        initial_time[block][trial] = clock_it.getTime()
                        x_center = mouse_x
                        y_center = mouse_y
                        first_flag = False
                    cursor_x, cursor_y = (mouse_x-x_center)*np.cos(rotation) - (mouse_y-y_center)*np.sin(rotation), (mouse_x-x_center)*np.sin(rotation) + (mouse_y-y_center)*np.cos(rotation)
                    #Change the position of the fixation to the rotated movement from the mouse
                    fixation.pos = (cursor_x, cursor_y)
                    traj_x.append(cursor_x)
                    traj_y.append(cursor_y)
                    #Draw on screen
                    fixation.draw()
                    target.draw()

                    #Display on Screen
                    my_win.flip()

                    #Break if fixation reaches the target
                    if visual.helpers.polygonsOverlap(fixation, target):
                        movement_time[block][trial] = clock_mt.getTime()
                        print("Broke")
                        fixation.pos = [0, 0]
                        fixation.draw()
                        #Store Trajectory
                        #print (traj_x)
                        rotated_traj_x[block][trial] = traj_x
                        #print ('Here')
                        #print (rotated_traj_x)
                        rotated_traj_y[block][trial] = traj_y
                        break
                    #print(rotated_traj_x)
                else:
                    fixation.pos = [0, 0]
                    fixation.draw()
                    target.draw()
                    my_mouse.setPos([0, 0])
                    my_win.flip()
                    
            #return fixation back to center
            fixation.fillColor = [0.5, 0.5, 0.5]
            fixation.lineColor = [0.5, 0.5, 0.5]
            fixation.pos = [0, 0]
            fixation.draw()
            my_win.flip()

    core.wait(1.0)

    actual_traj = np.array([rotated_traj_x, rotated_traj_y])
    #Cleanup
    my_win.close()
    #core.quit()
    return (target_angle_pos, actual_traj, initial_time, movement_time)



### Display

In [119]:
def rotated_blocks(num_blocks, num_trials, rotation):
    thetas = np.repeat(np.array([np.pi/4, 3*np.pi/4, 5*np.pi/4, 7*np.pi/4]), 4)
    target_pos_dict = {'Go':np.random.permutation(thetas), 'NoGo': np.random.permutation(thetas)}
    target_pos_df = pd.DataFrame(target_pos_dict)
    target_pos = target_pos_df.melt().take(np.random.permutation(32))
    target_pos = target_pos.append(target_pos_df.melt().take(np.random.permutation(32)), ignore_index = True)
    #Access using target_pos.iloc[i].value/var
    #target_pos_arrays = target_pos.to_numpy()
    target_pos_arrays = np.array(target_pos)
    target_pos_radius = 0.75
    #multiplication factor. ADjust according to screen dimensions
    target_xpos_rotated = 0.42*np.cos(target_pos_arrays[:, 1].astype(float))
    target_ypos_rotated = 0.75*np.sin(target_pos_arrays[:, 1].astype(float))
    
    
    # Create a window to draw in
    my_win = visual.Window((1920, 1080), allowGUI=True)
    
    
    
    #Initialize Variables to return
    rotated_traj_x = np.empty((num_blocks, num_trials), dtype = object)
    rotated_traj_y = np.empty((num_blocks, num_trials), dtype = object)
    initial_time = np.zeros((num_blocks, num_trials))
    movement_time = np.zeros((num_blocks, num_trials))
    #Center fixation
    fixation = visual.Circle(my_win, edges=64, size = (0.03, 0.05), pos = [0, 0], fillColor = [0.5, 0.5, 0.5], lineColor = [0.5, 0.5, 0.5])

    #Initialize mouse object
    for block in range(num_blocks):
        display_scoreboard(my_win, 'Your score: 10\n Tap Stylus to continue')
        for trial in range(num_trials):
            traj_x = list()
            traj_y = list()
            fixation.draw()
            my_win.update()
            #isi
            core.wait(np.random.normal(3.5, 1))
            
            target = visual.Circle(my_win, edges=64, size = (0.1, 0.17), pos = [target_xpos_rotated[trial], target_ypos_rotated[trial]], fillColor = [1, -1, -1], lineColor = [1, -1, -1])
            fixation.draw()
            target.draw()            
            my_win.update()
            
            
            #Wait for color change
            core.wait(np.random.uniform(1.5, 2.5))
            
            if target_pos_arrays[trial, 0] == 'NoGo':
                fixation.fillColor = [1, 0.32, 0.28] 
                fixation.lineColor = [1, 0.332, 0.28]
                fixation.draw()
                target.draw()
                my_win.update()
                core.wait(0.5, 1)
                fixation.fillColor = [0.5, 0.5, 0.5]
                fixation.lineColor = [0.5, 0.5, 0.5]
                fixation.pos = [0, 0]
                fixation.draw()
                my_win.flip()

                continue
                
            #Change fixation color
            fixation.fillColor = [-0.5, 1, 0.9] 
            fixation.lineColor = [-0.5, 1, 0.9]
            fixation.draw()
            target.draw()
            my_mouse = event.Mouse(visible=False,newPos=[0,0],win=my_win)
            my_win.update()
            clock_it = core.Clock()
            clock_mt = core.Clock()
            first_flag = True
            while True:
                #Get current mouse position
                mouse_x, mouse_y = my_mouse.getPos()
                    
                #print (mouse_x, mouse_y)
            #    if first == True:
            #        mouse_x, mouse_y = x_center, y_center
            #        first = False
                if my_mouse.getPressed()[0]:
                    if first_flag:
                        mouse_x, mouse_y = my_mouse.getPos()
                        initial_time[block][trial] = clock_it.getTime()
                        x_center = mouse_x
                        y_center = mouse_y
                        first_flag = False
                    cursor_x, cursor_y = (mouse_x-x_center)*np.cos(rotation) + (mouse_y-y_center)*np.sin(rotation), -(mouse_x-x_center)*np.sin(rotation) + (mouse_y-y_center)*np.cos(rotation)
                    #Change the position of the fixation to the rotated movement from the mouse
                    fixation.pos = (cursor_x, cursor_y)
                    traj_x.append(cursor_x)
                    traj_y.append(cursor_y)
                    #Draw on screen
                    fixation.draw()
                    target.draw()

                    #Display on Screen
                    my_win.flip()

                    #Break if fixation reaches the target
                    if visual.helpers.polygonsOverlap(fixation, target):
                        movement_time[block][trial] = clock_mt.getTime()
                        print("Broke")
                        fixation.pos = [0, 0]
                        fixation.draw()
                        #Store Trajectory
                        #print (traj_x)
                        rotated_traj_x[block][trial] = traj_x
                        #print ('Here')
                        #print (rotated_traj_x)
                        rotated_traj_y[block][trial] = traj_y
                        break
                    #print(rotated_traj_x)
                else:
                    fixation.pos = [0, 0]
                    fixation.draw()
                    target.draw()
                    my_mouse.setPos([0, 0])
                    my_win.flip()

                    
            #return fixation back to center
            fixation.fillColor = [0.5, 0.5, 0.5]
            fixation.lineColor = [0.5, 0.5, 0.5]
            fixation.pos = [0, 0]
            fixation.draw()
            my_win.flip()
            

    core.wait(1.0)
    actual_traj = np.array([rotated_traj_x, rotated_traj_y])
    #Cleanup
    my_win.close()
    #core.quit()
    return (target_pos_arrays, actual_traj, initial_time, movement_time)



## Transfer Blocks


### Target Stimuli

8180.7754 	WARNING 	Monitor specification not found. Creating a temporary one...


In [112]:
def transfer_blocks(num_blocks, num_trials, rotation):
    thetas = np.repeat(np.array([np.pi/4, 3*np.pi/4, 5*np.pi/4, 7*np.pi/4]), 4)
    target_pos_dict = {'Go':np.random.permutation(thetas), 'NoGo': np.random.permutation(thetas)}
    target_pos_df = pd.DataFrame(target_pos_dict)
    target_pos = target_pos_df.melt().take(np.random.permutation(32))
    target_pos = target_pos.append(target_pos_df.melt().take(np.random.permutation(32)), ignore_index = True)
    #Access using target_pos.iloc[i].value/var
    target_pos_arrays = target_pos.to_numpy()
    #target_pos_radius = 0.75
    target_xpos_rotated = 0.42*np.cos(target_pos_arrays[:, 1].astype(float))
    target_ypos_rotated = 0.75*np.sin(target_pos_arrays[:, 1].astype(float))
    
    
    # Create a window to draw in
    my_win = visual.Window((1920, 1080), allowGUI=True)
    rotated_traj_x = np.empty((num_blocks, num_trials), dtype = object)
    rotated_traj_y = np.empty((num_blocks, num_trials), dtype = object)
    initial_time = np.zeros((num_blocks, num_trials))
    movement_time = np.zeros((num_blocks, num_trials))
    #Center fixation
    fixation = visual.Circle(my_win, edges=64, size = (0.03, 0.05), pos = [0, 0], fillColor = [0.5, 0.5, 0.5], lineColor = [0.5, 0.5, 0.5])

    #Initialize mouse object
    for block in range(num_blocks):
        display_scoreboard(my_win, 'Your score: 10 \n Tap Stylus to continue.')
        for trial in range(num_trials):
            traj_x = list()
            traj_y = list()
            fixation.draw()
            my_win.update()
            #isi
            core.wait(np.random.normal(3.5, 1))
            
            target = visual.Circle(my_win, edges=64, size = (0.1, 0.17), pos = [target_xpos_rotated[trial], target_ypos_rotated[trial]], fillColor = [1, -1, -1], lineColor = [1, -1, -1])
            fixation.draw()
            target.draw()            
            my_win.update()
            
            
            #Wait for color change
            core.wait(np.random.uniform(1.5, 2.5))
            
            if target_pos_arrays[trial, 0] == 'NoGo':
                fixation.fillColor = [1, 0.32, 0.28] 
                fixation.lineColor = [1, 0.332, 0.28]
                fixation.draw()
                target.draw()
                my_win.update()
                core.wait(0.5, 1)
                fixation.fillColor = [0.5, 0.5, 0.5]
                fixation.lineColor = [0.5, 0.5, 0.5]
                fixation.pos = [0, 0]
                fixation.draw()
                my_win.flip()

                continue
                
            #Change fixation color
            fixation.fillColor = [-0.5, 1, 0.9] 
            fixation.lineColor = [-0.5, 1, 0.9]
            fixation.draw()
            target.draw()
            my_mouse = event.Mouse(visible=False,newPos=[0,0],win=my_win)
            my_win.update()
            clock_it = core.Clock()
            clock_mt = core.Clock()
            first_flag = True
            while True:
                #Get current mouse position
                mouse_x, mouse_y = my_mouse.getPos()
                    
                #print (mouse_x, mouse_y)
            #    if first == True:
            #        mouse_x, mouse_y = x_center, y_center
            #        first = False
                if my_mouse.getPressed()[0]:
                    if first_flag:
                        initial_time[block][trial] = clock_it.getTime()
                        x_center = mouse_x
                        y_center = mouse_y
                        first_flag = False
                    cursor_x, cursor_y = (mouse_x-x_center)*np.cos(rotation) - (mouse_y-y_center)*np.sin(rotation), -(mouse_x-x_center)*np.sin(rotation) + (mouse_y-y_center)*np.cos(rotation)
                    #Change the position of the fixation to the rotated movement from the mouse
                    fixation.pos = (cursor_x, cursor_y)
                    traj_x.append(cursor_x)
                    traj_y.append(cursor_y)
                    #Draw on screen
                    fixation.draw()
                    target.draw()

                    #Display on Screen
                    my_win.flip()

                    #Break if fixation reaches the target
                    if visual.helpers.polygonsOverlap(fixation, target):
                        movement_time[block][trial] = clock_mt.getTime()
                        print("Broke")
                        fixation.pos = [0, 0]
                        fixation.draw()
                        #Store Trajectory
                        #print (traj_x)
                        rotated_traj_x[block][trial] = traj_x
                        #print ('Here')
                        #print (rotated_traj_x)
                        rotated_traj_y[block][trial] = traj_y
                        break
                    #print(rotated_traj_x)
                else:
                    fixation.pos = [0, 0]
                    fixation.draw()
                    target.draw()
                    my_mouse.setPos([0, 0])
                    my_win.flip()
                    
            #return fixation back to center
            fixation.fillColor = [0.5, 0.5, 0.5]
            fixation.lineColor = [0.5, 0.5, 0.5]
            fixation.pos = [0, 0]
            fixation.draw()
            my_win.flip()
            

    core.wait(1.0)
    actual_traj = np.array([rotated_traj_x, rotated_traj_y])
    #Cleanup
    display_scoreboard('Thank you for your Participation!')
    core.wait(5.0)
    my_win.close()
    #core.quit()
    return (target_pos_arrays, actual_traj, initial_time, movement_time)



# Experiment

## Demo

In [87]:
my_win = visual.Window((1920, 1080), allowGUI=True)
text = "This is just a demo.\nTap Stylus to continue"
display_scoreboard(my_win, text)
text = "Drag the smaller circle to the target using your stylus.\nTap Stylus to continue"
display_scoreboard(my_win, text)
my_win.close()
garbage = template_blocks(1, 4, 0)

my_win = visual.Window((1920, 1080), allowGUI=True)
text = "One more demo.\nDrag the smaller circle to the target using your stylus.\nTap Stylus to continue"
display_scoreboard(my_win, text)
my_win.close()
garbage = template_blocks(1, 4, 0)

8495.4811 	WARNING 	Monitor specification not found. Creating a temporary one...
Broke
Broke
Broke
Broke
8498.5954 	WARNING 	Monitor specification not found. Creating a temporary one...
8524.1749 	WARNING 	Monitor specification not found. Creating a temporary one...
Broke
Broke
Broke
Broke
8529.4128 	WARNING 	Monitor specification not found. Creating a temporary one...


(array([6.28318531, 2.35619449, 3.92699082, 5.49778714, 5.49778714,
        0.        , 2.35619449, 0.        , 6.28318531, 3.92699082,
        1.57079633, 1.57079633, 0.78539816, 0.78539816, 4.71238898,
        4.71238898, 6.28318531, 0.        , 5.49778714, 4.71238898,
        0.78539816, 5.49778714, 6.28318531, 0.78539816, 1.57079633,
        3.92699082, 1.57079633, 2.35619449, 0.        , 4.71238898,
        3.92699082, 2.35619449, 4.71238898, 5.49778714, 1.57079633,
        3.92699082, 5.49778714, 1.57079633, 6.28318531, 0.        ,
        3.92699082, 6.28318531, 0.        , 2.35619449, 0.78539816,
        4.71238898, 0.78539816, 2.35619449, 5.49778714, 0.78539816,
        4.71238898, 1.57079633, 2.35619449, 0.        , 1.57079633,
        6.28318531, 3.92699082, 0.        , 0.78539816, 2.35619449,
        4.71238898, 3.92699082, 5.49778714, 6.28318531]),
 array([[[list([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.005208333333333333, 0.02083333333

7702.3489 	WARNING 	Monitor specification not found. Creating a temporary one...


## Run Template trials

In [110]:
template_targets, template_traj, template_it, template_mt = template_blocks(1, 2, 0)

Broke
Broke
12734.0124 	WARNING 	Monitor specification not found. Creating a temporary one...


## Display Instructions for the rest of the experiment

In [58]:
my_win = visual.Window((1920, 1080), allowGUI=True)
text = 'For the rest of the experiment, wait for the center fixation to change color.\nTap stylus to continue'
display_scoreboard(my_win, text)

text = 'Drag the center to the target when the center turns Blue.\n Tap stylus for a short demo'
display_scoreboard(my_win, text)


text = 'Do not drag the center to the target when the center turns Red.\n Tap stylus for a short demo'
display_scoreboard(my_win, text)
my_win.close()


garbage = transfer_blocks(1, 2, 0)

my_win = visual.Window((1920, 1080), allowGUI=True)
text = 'One more short demo. Tap stylus. Go when blue. Do not go when Red'
display_scoreboard(my_win, text)
garbage = transfer_blocks(1, 2, 0)

my_win.close()


7326.1587 	WARNING 	Monitor specification not found. Creating a temporary one...
Broke
7339.5674 	WARNING 	Monitor specification not found. Creating a temporary one...
Broke
7357.2514 	WARNING 	Monitor specification not found. Creating a temporary one...
7367.0520 	WARNING 	Monitor specification not found. Creating a temporary one...


## Rotated Blocks

In [120]:
rotated_targets, rotated_traj, rotated_it, rotated_mt = rotated_blocks(1, 2, 90)

Broke
Broke
17023.2265 	WARNING 	Monitor specification not found. Creating a temporary one...


## Transfer Blocks

In [105]:
transfer_targets, transfer_traj, transfer_it, transfer_mt = transfer_blocks(1, 2, 0)

Broke
12159.8421 	WARNING 	Monitor specification not found. Creating a temporary one...


## Save the data

In [88]:
import os

In [94]:
#make parent dir
parent_dir = 'data\\data'+ok_data[0]
os.mkdir(parent_dir)

In [102]:
#Directory for templates
template_directory = 'templates'
try:
    os.mkdir(os.path.join(parent_dir, template_directory))
except OSError as error:
    print ('Already exists')
np.save(os.path.join(parent_dir, template_directory, 'targets.npy'), template_targets)
np.save(os.path.join(parent_dir, template_directory, 'trajectories.npy'), template_traj)
np.save(os.path.join(parent_dir, template_directory, 'its.npy'), template_it)
np.save(os.path.join(parent_dir, template_directory, 'mts.npy'), template_mt)

Already exists


In [107]:
#Directory for Rotated
rotated_directory = 'rotated'
try:
    os.mkdir(os.path.join(parent_dir, rotated_directory))
except OSError as error:
    print ('Already exists')
np.save(os.path.join(parent_dir, rotated_directory, 'targets.npy'), rotated_targets)
np.save(os.path.join(parent_dir, rotated_directory, 'trajectories.npy'), rotated_traj)
np.save(os.path.join(parent_dir, rotated_directory, 'its.npy'), rotated_it)
np.save(os.path.join(parent_dir, rotated_directory, 'mts.npy'), rotated_mt)

Already exists


In [106]:
#Directory for Rotated
transfer_directory = 'transfer'
try:
    os.mkdir(os.path.join(parent_dir, transfer_directory))
except OSError as error:
    print ('Already exists')
np.save(os.path.join(parent_dir, transfer_directory, 'targets.npy'), transfer_targets)
np.save(os.path.join(parent_dir, transfer_directory, 'trajectories.npy'), transfer_traj)
np.save(os.path.join(parent_dir, transfer_directory, 'its.npy'), transfer_it)
np.save(os.path.join(parent_dir, transfer_directory, 'mts.npy'), transfer_mt)